In [1]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
data = pd.read_csv("Watermelon dataset.csv") 
print(data.head()) 


  Brand name                                       Product name  \
0       cleo  Cleo Glow Radiance Booster Lightening Day Flui...   
1       cleo  Cleo Glow Radiance Booster Lightening Night Fl...   
2       cleo  Cleo Glow Lightening Cleansing Gel Ultra Hydra...   
3       cleo  Cleo Glow Vitamin C Cleansing Gel (All Skin Ty...   
4       cleo                      Cleo Glow Vitamin C 10% Serum   

                                    Text Description Price Range  \
0  The Cleo Glow Radiance Booster Lightening Day ...     100-200   
1  Cleo Glow Radiance Booster Lightening Night Fl...     100-200   
2  The Cleo Glow Lightening Cleansing Gel is a ge...     100-200   
3  The Cleo Glow Vitamin C Cleansing Gel is a gen...     100-200   
4  Cleo Glow Vitamin C 10% Serum is a powerful an...     100-200   

                                           Image URL Country of manufacture  \
0  https://i0.wp.com/cleolaboratories.com/wp-cont...                  Egypt   
1  https://i0.wp.com/cleolabor

In [3]:
data['Text Description'] = data['Text Description'].apply(lambda x: x.lower())


In [4]:
import string
translator = str.maketrans('', '', string.punctuation) 
data['Text Description'] = data['Text Description'].apply(lambda x: x.translate(translator))


In [5]:
data['Text Description'] = data['Text Description'].apply(lambda x: ' '.join(x.split()))


In [6]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english')) 
data['Text Description'] = data['Text Description'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words]))


In [7]:
vectorizer = TfidfVectorizer()
tfidf_features = vectorizer.fit_transform(data['Text Description'])

X = np.hstack([tfidf_features.toarray(), data[['Price Range']]])  # Include 'Price' feature



In [8]:
from nltk.stem import PorterStemmer 

def preprocess_new_text(text):
    stemmer = PorterStemmer() 
    stop_words = set(stopwords.words('english'))

    text = text.lower()  
    text = text.translate(str.maketrans('', '', string.punctuation)) 
    text = ' '.join(text.split())  
    text = ' '.join([stemmer.stem(word) for word in text.split() if word not in stop_words]) 

    return text 


In [9]:
encoder = LabelEncoder()
y = encoder.fit_transform(data['Category '])

In [10]:
from keras.preprocessing.sequence import pad_sequences

max_length = 200 

tokenizer = Tokenizer()
tokenizer.fit_on_texts(data['Text Description'])
sequences = tokenizer.texts_to_sequences(data['Text Description'])

padded_sequences = pad_sequences(sequences, maxlen=max_length) 


In [11]:
X = tfidf_features.toarray().astype(np.float32)  



In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42) 

In [13]:
from sklearn.metrics.pairwise import cosine_similarity

def similarity_calculation(query_tfidf, tfidf_features):
    similarities = cosine_similarity(query_tfidf, tfidf_features)
    return similarities 



In [14]:
def suggest_products(user_query, num_suggestions=5):
    processed_query = preprocess_new_text(user_query)
    query_tfidf = vectorizer.transform([processed_query])

    similarities = similarity_calculation(query_tfidf, tfidf_features) 

    top_indices = similarities.argsort()[0][-num_suggestions:][::-1]
    suggested_products = data.iloc[top_indices]

    return suggested_products

user_query = "Looking for a hydrating face cream"
suggestions = suggest_products(user_query)
print(suggestions)

   Brand name                                       Product name  \
8        cleo  Cleo Youth Hyaluronic Acid Day Cream SPF 30 An...   
7        cleo  Cleo Youth Hyaluronic Acid+ Anti-Aging Night C...   
23       cleo                    CLEO GROW ENERGIZING HAIR CREAM   
25       cleo  Cleo Sun Hydrating Sunscreen Cream Broad Spect...   
6        cleo           Cleo Youth Hyaluronic Acid Cleansing Gel   

                                     Text Description Price Range  \
8   cleo youth hyaluronic acid day cream spf 30 an...     100-200   
7   cleo youth hyaluronic acid antiaging night cre...     100-200   
23  cleo grow energizing hair cream rich hair crea...     100-200   
25  cleo laboratories sun hydrating sunscreen crea...     100-200   
6   cleo youth hyaluronic acid cleansing gel deep ...     100-200   

                                            Image URL Country of manufacture  \
8   https://i0.wp.com/cleolaboratories.com/wp-cont...                  Egypt   
7   https://i0.w

In [37]:
import gradio as gr
import requests  

#def fetch_image(image_url):
 #   try:
  #      response = requests.get(image_url, stream=True)
   #     return response.raw
    #except requests.exceptions.RequestException:
     #   return None  
    
def display_product(brand, name, price):
    return f"""
           <div style='margin-bottom: 15px; border: 1px solid #eee'>
               <b>{brand}</b> - {name} <br>
               Price Range: {price}
           </div>
           """ 

    
def predict(user_query): 
    suggestions = suggest_products(user_query)
   
    suggestions_display = suggestions[['Brand name', 'Product name', 'Price Range']]
    suggestions_display = suggestions_display.drop_duplicates()

    return suggestions_display.to_html(escape=False, index=False) 
 
iface = gr.Interface(fn=predict, 
                     inputs="text",  
                     outputs=gr.HTML(), 
                     title="Watermelon Product Suggester",
                     description="Find your perfect skincare and more",
                     examples=["Looking for a gentle cleanser", "Need a moisturizer for oily skin"])

iface.launch() 

Running on local URL:  http://127.0.0.1:7877

To create a public link, set `share=True` in `launch()`.
